# Aprendizado Supervisionado II - Trabalho 3

## Pacotes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.svm import SVC

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, plot_confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Pré-Processamento

In [ ]:
voice= pd.read_csv('/kaggle/input/voicegender/voice.csv')

In [ ]:
voice.info()

In [ ]:
voice['label'].value_counts()

In [ ]:
voice.describe()

In [ ]:
voice.isnull().sum()

In [ ]:
voice['label'] = voice['label'].map({'male': 1, 'female': 0})
voice.head()

## Análise exploratória 

### Plots comparativos

Podemos observar que as distribuições das variáveis 'sd',  'Q25', 'IQR' e 'meanfun' nos histogramas uma menor intersecção entre as massas de dados e médias mais bem definidas para cada uma das duas categorias de voz. Para uma análise mais parcimoniosa, seria interessante tentar classificar as categorias de voz utilizando somente essas covariáveis e comparar com a classificação onde são utilizadas todas as variáveis.

In [ ]:
fig = plt.figure(figsize = (20, 15))
j = 0
for i in voice.columns:
    plt.subplot(5, 5, j+1)
    j += 1
    sns.distplot(voice[i][voice['label']==0], label = 'Female')
    sns.distplot(voice[i][voice['label']==1],  label = 'Male')
    plt.legend(loc='best')
fig.suptitle('Voice Data Analysis')
fig.tight_layout()
fig.subplots_adjust(top=0.90)
plt.show()

In [ ]:
sns.pairplot(voice[['sd', 'Q25', 'IQR', 'meanfun', 'label']], hue = 'label', size = 3)
plt.show()

### Correlação

Se observarmos as correlações entre as variáveis, vemos que as maiores correlações (superiores em módulo à 0.5) com a variável 'label', referente às categorias que estamos interessados em avaliar, são de 'Q25', 'IQR' e 'meanfun'. Associando essa informação com a oferecida observando os histogramas, temos uma boa confiança de que pelo menos as 3 variáveis 'Q25', 'IQR' e 'meanfun' são mais importantes para a análise do que as outras.

In [ ]:
#Correlação
_, ax = plt.subplots(figsize=(20, 20))
sns.heatmap(voice.corr(), annot=True, linewidths=.5, fmt= '.1f', ax=ax)
plt.show()

In [ ]:
voice.corr()

In [ ]:
voice.corr()['label'].apply(lambda x: abs(x)).sort_values(ascending=True).iloc[1:21][::-1]

## Classificadores

Foram utilizados seis classificadores nesse trabalho. Entre os classificadores paramétricos foram utilizadas três variações de Máquina de Vetor Suporte (SVM):  kernel linear, kernel radial e kernel radial com padronização escalar. Entre os classificadores não-paramétricos foram utilizados o K-nearest neighbors (KNN), Árvores de decisão e Bagging.

As 'label' foram reclassificadas como 'female' sendo '0' e 'male' sendo '1'.

As amostras foram divididas em 70% para treino e 30% para teste.

A escolha dos hiperparâmetros foi feita por meio de validação cruzada.

In [ ]:
voiceCopy = voice.copy()

In [ ]:
X = voiceCopy.drop(columns = ['label'])
y = voiceCopy['label']
results = []

In [ ]:
test_size = 0.3
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify = y, test_size = test_size)

### SVM kernel linear

Para a seleção do hiperparâmetro 'C' foi utilizada validação cruzada. Foram testados 10 hiperparâmetros em 5 grupos (5-fold cross-validation), totalizando 50 ajustes. O melhor parâmetro de regularização 'C' apontado por esse método de seleção foi 10.

Obtivemos 97% de precisão na classificação, com apenas 1% de casos em que identificamos a voz masculina como feminina (falso negativo) e 3% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
param_lin = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 150, 200]}]
svm_lin_CV = GridSearchCV(SVC(kernel = 'linear'), param_grid = param_lin, cv = 5, scoring = 'accuracy', verbose = 4, n_jobs = -1)

svm_lin_CV.fit(X_train, y_train)

In [ ]:
pd.DataFrame(svm_lin_CV.cv_results_)

In [ ]:
svm_lin_CV.best_estimator_

In [ ]:
y_test_pred_lin_CV = svm_lin_CV.predict(X_test)

print(metrics.confusion_matrix(y_test, y_test_pred_lin_CV))
print(metrics.accuracy_score(y_test, y_test_pred_lin_CV))
print(metrics.f1_score(y_test, y_test_pred_lin_CV))

In [ ]:
#SVM Kernel Linear
SVM_L = SVC(kernel = 'linear', C = 10)
SVM_L.fit(X_train, y_train)
y_pred_svm_l = SVM_L.predict(X_test)

CM = confusion_matrix(y_test, y_pred_svm_l)
acc_svm_l = accuracy_score(y_test, y_pred_svm_l)
#score = LDA.score(X_test, y_test)
score_svm_l = cross_val_score(SVM_L, X, y, cv=5) # cross-validation
results.append(acc_svm_l)

print("Score : ", score_svm_l.mean())
print("SVM Kernel Linear Accuracy: ", acc_svm_l)

plot_confusion_matrix(SVM_L, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(SVM_L, X_test, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test, y_pred_svm_l, target_names=target_names))

### SVM kernel radial

Para a seleção dos hiperparâmetros 'C' e 'gamma' foi utilizada validação cruzada. Foram testados 13 valores diferentes para 'C' e 9 valores para 'gamma' em 5 grupos (5-fold cross-validation), totalizando 585 ajustes. O melhor parâmetro de regularização 'C' apontado por esse método de seleção foi 100000 e gamma igual a 0.0001.

Obtivemos aproximadamente 96% de precisão na classificação, com apenas 2% de casos em que identificamos a voz masculina como feminina (falso negativo) e 2% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
param_rbf = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 1000, 5000, 10000, 50000, 100000], 
              'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5, 10, 50, 100]}] # por algum motivo, com rbf é mais rápido!

svm_rbf_CV = GridSearchCV(SVC(kernel = 'rbf'), param_grid = param_rbf, cv = 5, scoring = 'accuracy', verbose = 4, n_jobs = -1)

svm_rbf_CV.fit(X_train, y_train)

In [ ]:
pd.DataFrame(svm_rbf_CV.cv_results_)

In [ ]:
svm_rbf_CV.best_estimator_

In [ ]:
y_test_pred_rbf_CV = svm_rbf_CV.predict(X_test)

print(metrics.confusion_matrix(y_test, y_test_pred_rbf_CV))
print(metrics.accuracy_score(y_test, y_test_pred_rbf_CV))
print(metrics.f1_score(y_test, y_test_pred_rbf_CV))

In [ ]:
svm_rbf = SVC(kernel = 'rbf', gamma = 0.0001, C = 100000)
svm_rbf.fit(X_train, y_train)

y_test_pred_rbf = svm_rbf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_test_pred_rbf))
print(metrics.accuracy_score(y_test, y_test_pred_rbf))
print(metrics.f1_score(y_test, y_test_pred_rbf))

In [ ]:
#SVM radial
CM = confusion_matrix(y_test, y_test_pred_rbf)
acc_svm_rbf = accuracy_score(y_test, y_test_pred_rbf)
#score = LDA.score(X_test, y_test)
score_svm_rbf = cross_val_score(svm_rbf, X, y, cv=5) # cross-validation
results.append(acc_svm_rbf)

print("Score : ", score_svm_rbf.mean())
print("SVM Kernel Radial Accuracy: ", acc_svm_rbf)

plot_confusion_matrix(svm_rbf, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(svm_rbf, X_test, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test,y_test_pred_rbf, target_names=target_names))

### SVM kernel radial + StandardScaler

Para a seleção dos hiperparâmetros 'C' e 'gamma' foi utilizada validação cruzada. Foram testados 13 valores diferentes para 'C' e 8 valores para 'gamma' em 5 grupos (5-fold cross-validation), totalizando 520 ajustes. O melhor parâmetro de regularização 'C' apontado por esse método de seleção foi 10 e 'gamma' igual a 0.1.

Diferente do classificador radial apresentado anteriormente, aqui padronizamos os conjuntos de treino e teste referentes às características de interesse utilizadas para classificação.

Obtivemos aproximadamente 98% de precisão na classificação, com apenas 1% de casos em que identificamos a voz masculina como feminina (falso negativo) e 1% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

In [ ]:
param_rbf = [{'C': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100, 1000, 5000, 10000, 50000, 100000], 
              'gamma': [0.001, 0.01, 0.1, 1, 5, 10, 50, 100]}] # por algum motivo, com rbf é mais rápido!

svm_rbf_s_CV = GridSearchCV(SVC(kernel = 'rbf'), param_grid = param_rbf, cv = 5, scoring = 'accuracy', verbose = 4, n_jobs = -1)

svm_rbf_s_CV.fit(X_train_s, y_train)

In [ ]:
pd.DataFrame(svm_rbf_s_CV.cv_results_)

In [ ]:
svm_rbf_s_CV.best_estimator_

In [ ]:
y_test_pred_rbf_s_CV = svm_rbf_s_CV.predict(X_test_s)

print(metrics.confusion_matrix(y_test, y_test_pred_rbf_s_CV))
print(metrics.accuracy_score(y_test, y_test_pred_rbf_s_CV))
print(metrics.f1_score(y_test, y_test_pred_rbf_s_CV))

In [ ]:
svm_rbf_s = SVC(kernel = 'rbf', gamma = 0.1, C = 10)
svm_rbf_s.fit(X_train_s, y_train)

y_test_pred_rbf_s = svm_rbf_s.predict(X_test_s)

print(metrics.confusion_matrix(y_test, y_test_pred_rbf_s))
print(metrics.accuracy_score(y_test, y_test_pred_rbf_s))
print(metrics.f1_score(y_test, y_test_pred_rbf_s))

In [ ]:
X_s = scaler.fit_transform(X)

In [ ]:
#SVM radial+StandardScaler
CM = confusion_matrix(y_test, y_test_pred_rbf_s)
acc_svm_rbf_s = accuracy_score(y_test, y_test_pred_rbf_s)
#score = LDA.score(X_test, y_test)
score_svm_rbf_s = cross_val_score(svm_rbf_s, X_s, y, cv=5) # cross-validation
results.append(acc_svm_rbf_s)

print("Score : ", score_svm_rbf_s.mean())
print("SVM Kernel Radial + StandardScaler Accuracy: ", acc_svm_rbf_s)

plot_confusion_matrix(svm_rbf_s, X_test_s, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(svm_rbf_s, X_test_s, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test,y_test_pred_rbf_s, target_names=target_names))

### KNN

Para a seleção dos hiperparâmetros 'n_neighbors' e 'p' foi utilizada validação cruzada. Foram testados 6 valores diferentes para 'n_neighbors' e 4 valores para 'p' em 5 grupos (5-fold cross-validation), totalizando 520 ajustes. O melhor parâmetro de regularização 'n_neighbors' apontado por esse método de seleção foi 3 e 'p' igual a 1.

Obtivemos aproximadamente 77% de precisão na classificação, com 20% de casos em que identificamos a voz masculina como feminina (falso negativo) e 26% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
param_KNN = [{'n_neighbors': [1, 3, 5, 10, 15, 20], 
              'p': [1, 2, 3, 4]}]

KNN_CV = GridSearchCV(KNeighborsClassifier(), param_grid = param_KNN, cv = 5, 
                      scoring = 'accuracy', verbose = 4, n_jobs = -1)

KNN_CV.fit(X_train, y_train)

In [ ]:
pd.DataFrame(KNN_CV.cv_results_)

In [ ]:
KNN_CV.best_params_

In [ ]:
KNN = KNeighborsClassifier(n_neighbors = 3, p = 1, n_jobs = -1)
KNN.fit(X_train, y_train)

KNN_pred = KNN.predict(X_test)

print(metrics.confusion_matrix(y_test, KNN_pred))
print('Acurácia:', metrics.accuracy_score(y_test, KNN_pred))
print('F1:', metrics.f1_score(y_test, KNN.predict(X_test)))

#plot_clf(KNN, X_train, y_train, plot_test = True, X_test = X_test, y_test = y_test)

In [ ]:
#lda
CM = confusion_matrix(y_test, KNN_pred)
acc_knn = accuracy_score(y_test, KNN_pred)
#score = LDA.score(X_test, y_test)
score_knn = cross_val_score(KNN, X, y, cv=5) # cross-validation
results.append(acc_knn)

print("Score : ", score_knn.mean())
print("KNN Accuracy: ", acc_knn)

plot_confusion_matrix(KNN, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(KNN, X_test, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test,KNN_pred, target_names=target_names))

### Árvore de decisão

Para a seleção dos hiperparâmetros 'max_depth' e 'criterion' foi utilizada validação cruzada. Foram testados 9 valores diferentes para 'max_depth' e 2 valores para 'criterion' em 5 grupos (5-fold cross-validation), totalizando 90 ajustes. O melhor parâmetro de regularização 'max_depth' apontado por esse método de seleção foi 5 e 'criterion' igual a 'entropy'.

Obtivemos aproximadamente 96% de precisão na classificação, com 4% de casos em que identificamos a voz masculina como feminina (falso negativo) e 3% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
param_DT = [{'max_depth': [1, 3, 5, 7, 10, 13, 15, 18, 20], 
             'criterion': ['gini', 'entropy']}]

DT_CV = GridSearchCV(DecisionTreeClassifier(), param_grid = param_DT, cv = 5, 
                     scoring = 'accuracy', verbose = 4, n_jobs = -1)

DT_CV.fit(X_train, y_train)

In [ ]:
pd.DataFrame(DT_CV.cv_results_)

In [ ]:
DT_CV.best_params_

In [ ]:
print(metrics.confusion_matrix(y_test, DT_CV.predict(X_test)))
print(metrics.accuracy_score(y_test, DT_CV.predict(X_test)))
print(metrics.f1_score(y_test, DT_CV.predict(X_test)))

#plot_clf(DT_CV, X_train, y_train, plot_test = True, X_test = X_test, y_test = y_test)

In [ ]:
DT = DecisionTreeClassifier(max_depth = 3, criterion = 'gini',splitter='best')
DT.fit(X_train, y_train)

DT_pred = DT.predict(X_test)

print(metrics.confusion_matrix(y_test,DT_pred))
print(metrics.accuracy_score(y_test, DT_pred))
print(metrics.f1_score(y_test, DT_pred))

In [ ]:
#Decision Tree
CM = confusion_matrix(y_test, DT_pred)
acc_dt = accuracy_score(y_test, DT_pred)
#score = LDA.score(X_test, y_test)
score_dt = cross_val_score(DT, X, y, cv=5) # cross-validation
results.append(acc_dt)

print("Score : ", score_dt.mean())
print("Decision Tree Accuracy: ", acc_dt)

plot_confusion_matrix(DT, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(DT, X_test, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test,DT_pred, target_names=target_names))

In [ ]:
plt.figure(figsize = (20, 20))
plot_tree(DT)
plt.show()

### Bagging

Para a seleção dos hiperparâmetros 'base_estimator__max_depth', 'base_estimator__criterion' e 'max_samples' foi utilizada validação cruzada. Foram testados 4 valores diferentes para 'base_estimator__max_depth', 2 valores para 'base_estimator__criterion' e 6 valores para 'max_samples' em 5 grupos (5-fold cross-validation), totalizando 240 ajustes. O melhor parâmetro de regularização 'base_estimator__max_depth' apontado por esse método de seleção foi 15, 'base_estimator__criterion' igual a 'entropy' e 'max_samples' igual a 0.8.

Obtivemos aproximadamente 97% de precisão na classificação, com 3% de casos em que identificamos a voz masculina como feminina (falso negativo) e 2% de casos em que identificamos a voz feminina como masculina (falso positivo).

In [ ]:
# https://stackoverflow.com/questions/47570307/tuning-parameters-of-the-classifier-used-by-baggingclassifier

param_BAG = [{'base_estimator__max_depth': [7, 10, 13, 15], 
              'base_estimator__criterion': ['gini', 'entropy'], 
              'max_samples' : [0.05, 0.1, 0.2, 0.5, 0.8, 1]}]

BAG_CV = GridSearchCV(BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_jobs = -1), 
                      param_grid = param_BAG, cv = 5, scoring = 'accuracy', verbose = 4, n_jobs = -1)

BAG_CV.fit(X_train, y_train)

In [ ]:
pd.DataFrame(BAG_CV.cv_results_)

In [ ]:
BAG_CV.best_params_

In [ ]:
print(metrics.confusion_matrix(y_test, BAG_CV.predict(X_test)))
print(metrics.accuracy_score(y_test, BAG_CV.predict(X_test)))
print(metrics.f1_score(y_test, BAG_CV.predict(X_test)))

#plot_clf(BAG_CV, X_train, y_train, plot_test = True, X_test = X_test, y_test = y_test)

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html
# https://en.wikipedia.org/wiki/Bootstrap_aggregating

BAG = BaggingClassifier(base_estimator = DecisionTreeClassifier(max_depth = 10, criterion = 'entropy'), n_estimators = 100, max_samples= 0.5)
BAG.fit(X_train, y_train)

BAG_pred = BAG.predict(X_test)

print(metrics.confusion_matrix(y_test, BAG_pred))
print('Acurácia:', metrics.accuracy_score(y_test, BAG.predict(X_test)))
print('F1:', metrics.f1_score(y_test, BAG.predict(X_test)))

#plot_clf(BAG, X_train, y_train, plot_test = True, X_test = X_test, y_test = y_test)

In [ ]:
#Decision Tree
CM = confusion_matrix(y_test, BAG_pred)
acc_bag = accuracy_score(y_test, BAG_pred)
#score = LDA.score(X_test, y_test)
score_bag = cross_val_score(BAG, X, y, cv=5) # cross-validation
results.append(acc_bag)

print("Score : ", score_bag.mean())
print("Bagging Accuracy: ", acc_bag)

plot_confusion_matrix(BAG, X_test, y_test, cmap= "Blues")  
plt.show()

In [ ]:
metrics.plot_roc_curve(BAG, X_test, y_test)

In [ ]:
target_names = ['0','1']
print(classification_report(y_test,BAG_pred, target_names=target_names))

## Comparando

O modelo que apresentou a melhor acurácia foi o SVM radial+StandardScaler com 98.4%, sendo marginalmente melhor que o SVM com kernel linear e Bagging. Além disso apresentou score f1 de aproximadamente 0.95.

O plot de comparação entre a curva ROC de cada modelo mostra algumas características interessantes. Com exceção do KNN, a maioria dos classificadores têm AUC igual ou muito próxima a 1.

In [ ]:
print("SVM linear accuracy: ", acc_svm_l)

print("SVM radial accuracy: ",acc_svm_rbf)

print("SVM radial+StandardScaler accuracy: ",acc_svm_rbf_s)

print("KNN accuracy: ", acc_knn)

print("Decision Tree accuracy: ",acc_dt)

print("Bagging accuracy: ",acc_bag)

In [ ]:
print("SVM linear f1 score: ", score_svm_l.mean())

print("SVM radial f1 score: ",score_svm_rbf.mean())

print("SVM radial+StandardScaler f1 score: ",score_svm_rbf_s.mean())

print("KNN f1 score: ", score_knn.mean())

print("Decision Tree f1 score: ",score_dt.mean())

print("Bagging f1 score: ",score_bag.mean())

In [ ]:
print(cross_val_score(SVM_L, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(svm_rbf, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(svm_rbf_s, X_s, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(KNN, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(DT, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

print(cross_val_score(BAG, X, y, cv = 5,
                scoring = metrics.make_scorer(metrics.f1_score, greater_is_better = True, needs_proba = False)).mean())

In [ ]:
plt.figure(figsize=(10,8))

fig1 = metrics.plot_roc_curve(SVM_L, X_test, y_test)

fig2 = metrics.plot_roc_curve(svm_rbf, X_test, y_test,ax=fig1.ax_)
fig2.figure_.suptitle("ROC curve comparison")

fig3 = metrics.plot_roc_curve(svm_rbf_s, X_test_s, y_test,ax=fig1.ax_)
fig3.figure_.suptitle("ROC curve comparison")

fig4 = metrics.plot_roc_curve(KNN, X_test, y_test,ax=fig1.ax_)
fig4.figure_.suptitle("ROC curve comparison")

fig5 = metrics.plot_roc_curve(DT, X_test, y_test,ax=fig1.ax_)
fig5.figure_.suptitle("ROC curve comparison")

fig6 = metrics.plot_roc_curve(BAG, X_test, y_test,ax=fig1.ax_)
fig6.figure_.suptitle("ROC curve comparison")

plt.legend(loc=0)

## Conclusão

A análise exploratória dos dados de voz mostrou que as variáveis 'Q25', 'IQR' e 'meanfun' são as que distinguem melhor entre as vozes masculinas e femininas. Ao observarmos os histogramas podemos ver que essas variáveis apresentam médias mais distintas e a massa de dados menos coincidente em cada uma das curvas. A matriz de correlação mostra que essas variáveis também são as que possuem a maior correlação absoluta com as categorias de interesse. Talvez fosse interessante selecionar e testar essas variáveis em um modelo mais parcimonioso, a fim de melhorar a velocidade de processamento.

Ao analizarmos os seis classificadores observamos bons resultados para a maioria deles. Com excessão do K-nearest neighbours com precisão 77% , todos os outros apresentaram precisões e score f1 a cima de 95%, indicando que todos poderiam ser usados para classficação do tipo de voz sem muita diferença. A curva ROC também apresentou desempenho semelhante, apontando apenas o KNN como ruim.

Finalmente, o modelo escolhido como melhor em termos de acurácia para classificação do tipo de voz foi o SVM com kernel radial + StandardScaler, mas apenas com uma diferença marginal para o Bagging e o SVM com kernel linear.

### "Nota-se que treinar SVM com kernel RBF é significativamente mais rápido do que com o kernel linear. Tente explicar o motivo para tal, fazendo buscas na Internet se necessário".

A maioria das informações sobre o desempenho do svm nas comunidades de data science comentam apenas a cerca da complexidade dos cálculos realizados, apontados como de ordem O(n_samples^2 * n_features). Quando se busca por infomações sobre o desempenho específico do SVC(kernel='linear'), muitos comentarios (inclusive na própria página do scikit-learn) recomendam utilizar o sklearn.svm.LinearSVC como substituto por ser extremamente mais rápido, mas não comentam o contraste entre a velocidade de processamente dos diferentes kernels.

Referências:
https://scikit-learn.org/stable/modules/svm.html#complexity
https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html#sklearn.svm.LinearSVC
https://stackoverflow.com/questions/40077432/why-is-scikit-learn-svm-svc-extremely-slow
https://datascience.stackexchange.com/questions/989/svm-using-scikit-learn-runs-endlessly-and-never-completes-execution
https://www.csie.ntu.edu.tw/~cjlin/liblinear/
https://towardsdatascience.com/the-kernel-trick-c98cdbcaeb3f

## Apêndice: Testando classificadores utilizando apenas as variáveis com correlação superior a 0.5

In [ ]:
voice2 = voice[[ 'Q25', 'IQR', 'meanfun', 'label']].copy()

In [ ]:
X2 = voice2.drop(columns = ['label'])
y2 = voice2['label']
results = []

In [ ]:
test_size = 0.3
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,stratify = y2, test_size = test_size)

Quando apenas as variáveis com alta correlação são usada, o ajuste do KNN se torna muito melhor que o ajuste feito anteriormente. Os outros classificadores continuam tendo precisões semelhantes.

In [ ]:
param_KNN2 = [{'n_neighbors': [1, 3, 5, 10, 15, 20], 
              'p': [1, 2, 3, 4]}]

KNN2_CV = GridSearchCV(KNeighborsClassifier(), param_grid = param_KNN2, cv = 5, 
                      scoring = 'accuracy', verbose = 4, n_jobs = -1)

KNN2_CV.fit(X2_train, y2_train)

In [ ]:
KNN2_CV.best_params_

In [ ]:
KNN2 = KNeighborsClassifier(n_neighbors = 10, p = 2, n_jobs = -1)
KNN2.fit(X2_train, y2_train)

print('Acurácia:', metrics.accuracy_score(y2_test, KNN2.predict(X2_test)))
print('F1:', metrics.f1_score(y2_test, KNN2.predict(X2_test)))

In [ ]:
print(metrics.confusion_matrix(y2_test, KNN2.predict(X2_test)))

In [ ]:
DT2 = DecisionTreeClassifier(max_depth = 13, criterion = 'entropy',splitter='best')
DT2.fit(X2_train, y2_train)

DT2_pred = DT2.predict(X2_test)

print(metrics.confusion_matrix(y2_test,DT2_pred))
print(metrics.accuracy_score(y2_test, DT2_pred))
print(metrics.f1_score(y2_test, DT2_pred))

In [ ]:
param_BAG2 = [{'base_estimator__max_depth': [7, 10, 13, 15], 
              'base_estimator__criterion': ['gini', 'entropy'], 
              'max_samples' : [0.05, 0.1, 0.2, 0.5, 0.8, 1]}]

BAG2_CV = GridSearchCV(BaggingClassifier(base_estimator = DecisionTreeClassifier(), n_jobs = -1), 
                      param_grid = param_BAG2, cv = 5, scoring = 'accuracy', verbose = 4, n_jobs = -1)

BAG2_CV.fit(X2_train, y2_train)

In [ ]:
BAG2_CV.best_params_

In [ ]:
BAG2 = BaggingClassifier(base_estimator = DecisionTreeClassifier(max_depth = 10, criterion = 'entropy'), n_estimators = 100, max_samples= 0.8)
BAG2.fit(X2_train, y2_train)

BAG2_pred = BAG2.predict(X2_test)

print(metrics.confusion_matrix(y2_test, BAG2_pred))
print('Acurácia:', metrics.accuracy_score(y2_test, BAG2.predict(X2_test)))
print('F1:', metrics.f1_score(y2_test, BAG2.predict(X2_test)))

In [ ]:
def plot_clf(clf, X, y, h=0.02, pad=0.25, plot_test = False, X_test = None, y_test = None):
    x_min, x_max = X[:, 0].min()-pad, X[:, 0].max()+pad
    y_min, y_max = X[:, 1].min()-pad, X[:, 1].max()+pad
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    if plot_test == False:
        plt.figure(figsize = (6, 6))
        
        plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=plt.cm.Paired)
        plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.xlabel('X1')
        plt.ylabel('X2')
    else:
        plt.figure(figsize = (14, 6))
        
        plt.subplot(1, 2, 1)
        plt.scatter(X[:,0], X[:,1], s=70, c=y, cmap=plt.cm.Paired)
        plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.xlabel('X1')
        plt.ylabel('X2')
        
        plt.subplot(1, 2, 2)
        plt.scatter(X_test[:,0], X_test[:,1], s=70, c=y_test, cmap=plt.cm.Paired)
        plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.2)
        plt.xlim(x_min, x_max)
        plt.ylim(y_min, y_max)
        plt.xlabel('X1')
        plt.ylabel('X2')

    plt.show()